# EVM logs decode

In [ ]:
#!pip install web3

In [1]:
import json
import pandas as pd
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.logs import STRICT, IGNORE, DISCARD, WARN
from hexbytes import HexBytes
from web3.datastructures import AttributeDict
from eth_abi.codec import ABICodec
from eth_abi.registry import registry as default_registry
from eth_utils import (encode_hex, event_abi_to_log_topic)
from web3._utils.events import get_event_data
from web3.exceptions import (InvalidEventABI, LogTopicError, MismatchedABI)

default_codec = ABICodec(default_registry)

In [2]:
def get_event_abi(abi, abi_name, abi_type = 'event'):
    l = [x for x in abi if x['type'] == abi_type and x['name']==abi_name]
    return l[0]

In [3]:
w3 = Web3(Web3.HTTPProvider('https://polygon-rpc.com/', request_kwargs={'timeout': 10}))

# inject the poa compatibility middleware to the innermost layer
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

abi_str='''
[{"inputs":[{"internalType":"uint32","name":"_maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"_reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"_microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerTransmission","type":"uint32"},{"internalType":"address","name":"_link","type":"address"},{"internalType":"address","name":"_validator","type":"address"},{"internalType":"int192","name":"_minAnswer","type":"int192"},{"internalType":"int192","name":"_maxAnswer","type":"int192"},{"internalType":"contract AccessControllerInterface","name":"_billingAccessController","type":"address"},{"internalType":"contract AccessControllerInterface","name":"_requesterAccessController","type":"address"},{"internalType":"uint8","name":"_decimals","type":"uint8"},{"internalType":"string","name":"description","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"user","type":"address"}],"name":"AddedAccess","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"int256","name":"current","type":"int256"},{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"updatedAt","type":"uint256"}],"name":"AnswerUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"contract AccessControllerInterface","name":"old","type":"address"},{"indexed":false,"internalType":"contract AccessControllerInterface","name":"current","type":"address"}],"name":"BillingAccessControllerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint32","name":"maximumGasPrice","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"reasonableGasPrice","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"microLinkPerEth","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"linkGweiPerObservation","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"linkGweiPerTransmission","type":"uint32"}],"name":"BillingSet","type":"event"},{"anonymous":false,"inputs":[],"name":"CheckAccessDisabled","type":"event"},{"anonymous":false,"inputs":[],"name":"CheckAccessEnabled","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint32","name":"previousConfigBlockNumber","type":"uint32"},{"indexed":false,"internalType":"uint64","name":"configCount","type":"uint64"},{"indexed":false,"internalType":"address[]","name":"signers","type":"address[]"},{"indexed":false,"internalType":"address[]","name":"transmitters","type":"address[]"},{"indexed":false,"internalType":"uint8","name":"threshold","type":"uint8"},{"indexed":false,"internalType":"uint64","name":"encodedConfigVersion","type":"uint64"},{"indexed":false,"internalType":"bytes","name":"encoded","type":"bytes"}],"name":"ConfigSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":true,"internalType":"address","name":"startedBy","type":"address"},{"indexed":false,"internalType":"uint256","name":"startedAt","type":"uint256"}],"name":"NewRound","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint32","name":"aggregatorRoundId","type":"uint32"},{"indexed":false,"internalType":"int192","name":"answer","type":"int192"},{"indexed":false,"internalType":"address","name":"transmitter","type":"address"},{"indexed":false,"internalType":"int192[]","name":"observations","type":"int192[]"},{"indexed":false,"internalType":"bytes","name":"observers","type":"bytes"},{"indexed":false,"internalType":"bytes32","name":"rawReportContext","type":"bytes32"}],"name":"NewTransmission","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"transmitter","type":"address"},{"indexed":false,"internalType":"address","name":"payee","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"OraclePaid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"transmitter","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"},{"indexed":true,"internalType":"address","name":"proposed","type":"address"}],"name":"PayeeshipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"transmitter","type":"address"},{"indexed":true,"internalType":"address","name":"previous","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"}],"name":"PayeeshipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"user","type":"address"}],"name":"RemovedAccess","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"contract AccessControllerInterface","name":"old","type":"address"},{"indexed":false,"internalType":"contract AccessControllerInterface","name":"current","type":"address"}],"name":"RequesterAccessControllerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"requester","type":"address"},{"indexed":false,"internalType":"bytes16","name":"configDigest","type":"bytes16"},{"indexed":false,"internalType":"uint32","name":"epoch","type":"uint32"},{"indexed":false,"internalType":"uint8","name":"round","type":"uint8"}],"name":"RoundRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previous","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"}],"name":"ValidatorUpdated","type":"event"},{"inputs":[],"name":"LINK","outputs":[{"internalType":"contract LinkTokenInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"acceptOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"acceptPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"}],"name":"addAccess","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"billingAccessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"checkEnabled","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"disableAccessCheck","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"enableAccessCheck","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBilling","outputs":[{"internalType":"uint32","name":"maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"linkGweiPerTransmission","type":"uint32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"},{"internalType":"bytes","name":"_calldata","type":"bytes"}],"name":"hasAccess","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestConfigDetails","outputs":[{"internalType":"uint32","name":"configCount","type":"uint32"},{"internalType":"uint32","name":"blockNumber","type":"uint32"},{"internalType":"bytes16","name":"configDigest","type":"bytes16"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRound","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTransmissionDetails","outputs":[{"internalType":"bytes16","name":"configDigest","type":"bytes16"},{"internalType":"uint32","name":"epoch","type":"uint32"},{"internalType":"uint8","name":"round","type":"uint8"},{"internalType":"int192","name":"latestAnswer","type":"int192"},{"internalType":"uint64","name":"latestTimestamp","type":"uint64"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"linkAvailableForPayment","outputs":[{"internalType":"int256","name":"availableBalance","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"maxAnswer","outputs":[{"internalType":"int192","name":"","type":"int192"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"minAnswer","outputs":[{"internalType":"int192","name":"","type":"int192"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_signerOrTransmitter","type":"address"}],"name":"oracleObservationCount","outputs":[{"internalType":"uint16","name":"","type":"uint16"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"owedPayment","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address payable","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"}],"name":"removeAccess","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"requestNewRound","outputs":[{"internalType":"uint80","name":"","type":"uint80"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"requesterAccessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint32","name":"_maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"_reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"_microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerTransmission","type":"uint32"}],"name":"setBilling","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract AccessControllerInterface","name":"_billingAccessController","type":"address"}],"name":"setBillingAccessController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"_signers","type":"address[]"},{"internalType":"address[]","name":"_transmitters","type":"address[]"},{"internalType":"uint8","name":"_threshold","type":"uint8"},{"internalType":"uint64","name":"_encodedConfigVersion","type":"uint64"},{"internalType":"bytes","name":"_encoded","type":"bytes"}],"name":"setConfig","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"_transmitters","type":"address[]"},{"internalType":"address[]","name":"_payees","type":"address[]"}],"name":"setPayees","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract AccessControllerInterface","name":"_requesterAccessController","type":"address"}],"name":"setRequesterAccessController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_newValidator","type":"address"}],"name":"setValidator","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_to","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"},{"internalType":"address","name":"_proposed","type":"address"}],"name":"transferPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes","name":"_report","type":"bytes"},{"internalType":"bytes32[]","name":"_rs","type":"bytes32[]"},{"internalType":"bytes32[]","name":"_ss","type":"bytes32[]"},{"internalType":"bytes32","name":"_rawVs","type":"bytes32"}],"name":"transmit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"transmitters","outputs":[{"internalType":"address[]","name":"","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"validator","outputs":[{"internalType":"contract AggregatorValidatorInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_recipient","type":"address"},{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"withdrawFunds","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"withdrawPayment","outputs":[],"stateMutability":"nonpayable","type":"function"}]
'''
abi = json.loads(abi_str)

## <font color='blue'> 1. If we have a transaction hash </font>

In [4]:
txhash = '0x6bf6b3a80b09fada1a9bb07b1ad62d27074f0b763f041c7d0d98adbcd84e959b'
contract_address = '0xe41b5d02e64b165e77f12b72bf80b56d076000cf'

contract_address = Web3.toChecksumAddress(contract_address)
Contract = w3.eth.contract(address=contract_address, abi=abi)

receipt = w3.eth.getTransactionReceipt(txhash)
log = Contract.events.NewTransmission().processReceipt(receipt,errors=DISCARD) #errors=WARN

In [5]:
log[0]['args'] #log data

AttributeDict({'aggregatorRoundId': 601899,
 'answer': 288028518954,
 'transmitter': '0xd0A8Cb58efcee1CAeE48F3c357074862cA8210dc',
 'observations': [287449973102,
  287740000000,
  287874000000,
  287874000000,
  288003015405,
  288028518954,
  288028518954,
  288028518954,
  288028518954,
  288039486674,
  288325243765,
  288330727625,
  288621968577,
  288621968577,
  288673000000,
  288673000000],
 'observers': b'\x0e\x02\x0f\x04\x0c\x01\x03\x08\x00\t\x07\x06\x0b\r\n\x05',
 'rawReportContext': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x93t\x13\xae\xdb\x18\xe5v\x17J\xbf\xf5\x0e\x80\xdcF\x00\x02$\x0b\x01'})

In [6]:
log # whole log entry

(AttributeDict({'args': AttributeDict({'aggregatorRoundId': 601899,
   'answer': 288028518954,
   'transmitter': '0xd0A8Cb58efcee1CAeE48F3c357074862cA8210dc',
   'observations': [287449973102,
    287740000000,
    287874000000,
    287874000000,
    288003015405,
    288028518954,
    288028518954,
    288028518954,
    288028518954,
    288039486674,
    288325243765,
    288330727625,
    288621968577,
    288621968577,
    288673000000,
    288673000000],
   'observers': b'\x0e\x02\x0f\x04\x0c\x01\x03\x08\x00\t\x07\x06\x0b\r\n\x05',
   'rawReportContext': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x93t\x13\xae\xdb\x18\xe5v\x17J\xbf\xf5\x0e\x80\xdcF\x00\x02$\x0b\x01'}),
  'event': 'NewTransmission',
  'logIndex': 154,
  'transactionIndex': 18,
  'transactionHash': HexBytes('0x6bf6b3a80b09fada1a9bb07b1ad62d27074f0b763f041c7d0d98adbcd84e959b'),
  'address': '0xE41B5D02E64b165e77f12b72Bf80B56d076000CF',
  'blockHash': HexBytes('0x877dc840bc8b0b34418ff0b54e0e29f872bd1d011f8c684eef68

## <font color='blue'> 2. If we need to loop over all the logs for a partuclar contract </font>

In [7]:
NewTransmission_ABI = get_event_abi(abi, 'NewTransmission')
assert len(NewTransmission_ABI) > 0
NewTransmission_ABI #abi of the event                                                                                                             

{'anonymous': False,
 'inputs': [{'indexed': True,
   'internalType': 'uint32',
   'name': 'aggregatorRoundId',
   'type': 'uint32'},
  {'indexed': False,
   'internalType': 'int192',
   'name': 'answer',
   'type': 'int192'},
  {'indexed': False,
   'internalType': 'address',
   'name': 'transmitter',
   'type': 'address'},
  {'indexed': False,
   'internalType': 'int192[]',
   'name': 'observations',
   'type': 'int192[]'},
  {'indexed': False,
   'internalType': 'bytes',
   'name': 'observers',
   'type': 'bytes'},
  {'indexed': False,
   'internalType': 'bytes32',
   'name': 'rawReportContext',
   'type': 'bytes32'}],
 'name': 'NewTransmission',
 'type': 'event'}

In [8]:
#pull all the logs from an address

contract_address = '0xe41b5d02e64b165e77f12b72bf80b56d076000cf'
contract_address = Web3.toChecksumAddress(contract_address)

logs = w3.eth.get_logs({"address": contract_address
                       ,"topics":["0xf6a97944f31ea060dfde0566e4167c1a1082551e64b60ecb14d599a9d023d451"] #optional if we suppress mismatch errors
                       ,"fromBlock": w3.eth.get_block('latest')['number'] - 1000
                       ,"toBlock": 'latest'
                       })

all_events = []
for l in logs:
    try:
        evt = get_event_data(w3.codec, NewTransmission_ABI, l)
    except MismatchedABI: #if for some reason there are other events according to defined filter
        pass
    all_events.append(evt)
df = pd.DataFrame(all_events)
df

,address,args,blockHash,blockNumber,event,logIndex,transactionHash,transactionIndex
0,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...","b'\xf1\x98\x85""GX\x89\xceW*\x0b\xee\xb0\xf8QB\...",21670071,NewTransmission,143,b'\xd3\xef^\xda\xf1\xd6\xd5\xb6\t\xd02\x0f\xba...,31
1,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'\xb7HJG\x16\xff\xe2>\r\x8f\x8c\x8c\xb8q\xcbl...,21670076,NewTransmission,88,b'M\xb4\xbf\xd9\xa9\x89&\xf3\xb3\x81\x95\x107=...,19
2,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'\xe9[K\x17\x15\x8c\xad\x119\xa5l\x1d\x1c\xfa...,21670080,NewTransmission,573,b'$\xad\xbb\xc4\xc0\xe9\x00[\x85\xb6\xd7\xcd\x...,90
3,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'\xe7\x1ccW\xfcz\xce*U\xcb\x1b#\xd2\xf6\xc7\x...,21670094,NewTransmission,383,b'\xd12?\x81\xfd=[3\x84[\x1a\x8a\xa5\x8b\x82\x...,142
4,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'^\x8dV\x8c\xa1/\xaa\x87\x87\xafv7\x0b\x11\xc...,21670099,NewTransmission,38,b'z\xeb^T\x8d\x14\x1fm`Y\xc2t\x1az7P\x0fh\xc6d...,18
...,...,...,...,...,...,...,...,...
179,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'\x81\x96\xf3\xaafF\xba<\x0e8\xbb\xf0oI\xa8\x...,21671044,NewTransmission,141,b'\xba\xa4\xd8\x80\x9a\xee\xf5&w\x83Wy\xfc\xa2...,15
180,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'V\x1f\x8d\x0e\xc3\x7f}\x945\xd1\xc1l\x9a\nWe...,21671049,NewTransmission,67,b'{>\xcd\xd87\xddo\x8f\x112\x91\xf3\x99\x1b\xc...,10
181,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'\xbcU\xe64\x1d\xa9\x19(\xe2\x87\n.\xd5\x96\x...,21671054,NewTransmission,227,b'q\xc1\xd2A\x81H\xe9\xad\xee$xZL\xf9\xaa\xa5i...,23
182,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b's\xa8\xd8\xd4n\x83\xf2\xc7\xc7\x81A\xbe\xecS...,21671060,NewTransmission,260,"b""\x1f\xf9\x10\x04c\x15H\x90\xf4Vj6W\x9c\x0f\x...",28


In [9]:
df.loc[0,'args'] #parsed log data

AttributeDict({'aggregatorRoundId': 604266,
 'answer': 290031039784,
 'transmitter': '0x983D0e1d23D3109D15e833fB800A51ba154DEfD3',
 'observations': [289592242246,
  289601124494,
  289610000000,
  289648054131,
  289931277121,
  289934444470,
  289982973488,
  290031039784,
  290031039784,
  290081509673,
  290081509673,
  290081509673,
  290081509673,
  290088437772,
  290273353289,
  290550000000],
 'observers': b'\x04\x0f\x02\x0e\x06\x03\x0c\x0b\x07\x08\t\x01\x00\r\x05\n',
 'rawReportContext': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x93t\x13\xae\xdb\x18\xe5v\x17J\xbf\xf5\x0e\x80\xdcF\x00\x02&\xa3\x02'})

## <font color='blue'> 3. If we have only event data (no RPC conection required)</font>

### <font size = 3>NewTransmission</font>

In [10]:
#data from database
NewTransmission_data = '000000000000000000000000000000000000000000000000000000582f64b4720000000000000000000000005565b5362ff9f468ba2f144f38b87187c9a010a800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000002c0000000000000000000000066c8885c7645f4815b0bd9981dc827ed00000f900500000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000005810b9f3fd000000000000000000000000000000000000000000000000000000581b70c05b000000000000000000000000000000000000000000000000000000581b87b2c0000000000000000000000000000000000000000000000000000000581d3524990000000000000000000000000000000000000000000000000000005821ce44b3000000000000000000000000000000000000000000000000000000582763a9d2000000000000000000000000000000000000000000000000000000582763a9d2000000000000000000000000000000000000000000000000000000582763a9d2000000000000000000000000000000000000000000000000000000582f64b47200000000000000000000000000000000000000000000000000000058302628c8000000000000000000000000000000000000000000000000000000583765bf13000000000000000000000000000000000000000000000000000000583765bf13000000000000000000000000000000000000000000000000000000583b23b09a000000000000000000000000000000000000000000000000000000583dfe1f170000000000000000000000000000000000000000000000000000005846ea6bc1000000000000000000000000000000000000000000000000000000585dab5cb50000000000000000000000000000000000000000000000000000000000000010070b0f0500040c0a0201080e030d060900000000000000000000000000000000'

NewTransmission_ABI = get_event_abi(abi, 'NewTransmission')
NewTransmission_topic = encode_hex(event_abi_to_log_topic(NewTransmission_ABI)) 
NewTransmission_event = AttributeDict({'data':NewTransmission_data
                                      ,'topics': [HexBytes(NewTransmission_topic)
                                                 ,HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001')
                                                 ]
                                       ,'address':'' #'0x908edc7e1974ecab1ca7164424bc4cac287d83ad'
                                       ,'logIndex':''
                                       ,'transactionIndex':''
                                       ,'transactionHash':''
                                       ,'blockHash':''
                                       ,'blockNumber':''
                             
    })
get_event_data(default_codec, NewTransmission_ABI, NewTransmission_event) # w3.codec->default_codec if we have no rpc connection

AttributeDict({'args': AttributeDict({'aggregatorRoundId': 1,
  'answer': 378752250994,
  'transmitter': '0x5565b5362FF9f468bA2f144f38b87187C9a010A8',
  'observations': [378237744125,
   378417496155,
   378419000000,
   378447144089,
   378524288179,
   378617965010,
   378617965010,
   378617965010,
   378752250994,
   378764929224,
   378886536979,
   378886536979,
   378949316762,
   378997186327,
   379146890177,
   379528633525],
  'observers': b'\x07\x0b\x0f\x05\x00\x04\x0c\n\x02\x01\x08\x0e\x03\r\x06\t',
  'rawReportContext': b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00f\xc8\x88\\vE\xf4\x81[\x0b\xd9\x98\x1d\xc8'\xed\x00\x00\x0f\x90\x05"}),
 'event': 'NewTransmission',
 'logIndex': '',
 'transactionIndex': '',
 'transactionHash': '',
 'address': '',
 'blockHash': '',
 'blockNumber': ''})

### <font size = 3>BillingSet</font> 

In [11]:
data = '00000000000000000000000000000000000000000000000000000000000007d0000000000000000000000000000000000000000000000000000000000000006400000000000000000000000000000000000000000000000000000000055484680000000000000000000000000000000000000000000000000000000001ab3f00000000000000000000000000000000000000000000000000000000000a21fe80'

BillingSet_ABI = get_event_abi(abi, 'BillingSet')
BillingSet_topic = encode_hex(event_abi_to_log_topic(BillingSet_ABI)) 
BillingSet_event = AttributeDict({'data':data
                                 ,'topics': [HexBytes(BillingSet_topic)]
                                 ,'address':'' 
                                 ,'logIndex':''
                                 ,'transactionIndex':''
                                 ,'transactionHash':''
                                 ,'blockHash':''
                                 ,'blockNumber':''
                             
    })
get_event_data(default_codec, BillingSet_ABI, BillingSet_event) # w3.codec->default_codec if we have no rpc connection


AttributeDict({'args': AttributeDict({'maximumGasPrice': 2000,
  'reasonableGasPrice': 100,
  'microLinkPerEth': 89425000,
  'linkGweiPerObservation': 28000000,
  'linkGweiPerTransmission': 170000000}),
 'event': 'BillingSet',
 'logIndex': '',
 'transactionIndex': '',
 'transactionHash': '',
 'address': '',
 'blockHash': '',
 'blockNumber': ''})

### <font size = 3>ConfigSet</font> 

In [12]:
ConfigSet_data = '0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000e000000000000000000000000000000000000000000000000000000000000003000000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000005200000000000000000000000000000000000000000000000000000000000000010000000000000000000000000c01465eba4fa3a72309374bb67149a8fd14cb6870000000000000000000000008d766c898fa75c6a2fc59a078e8c2d2de12547e2000000000000000000000000cc1b49b86f79c7e50e294d3e3734fe94db9a42f00000000000000000000000008eb664cd767f12507e7e3864ba5b7e925090a0e50000000000000000000000003650da40fe97a93bfc2623e0dca3899a91eca0e20000000000000000000000000ac6c28b582016a55f6d4e3ac77b64749568ffe10000000000000000000000008316e3eb7eccfcaf0c1967903cca8ecda3df37e00000000000000000000000001e15545b23b831fd39e1d9579427dea61425dd470000000000000000000000005007b477f939646b4e4416afcef6b00567f5f078000000000000000000000000bd34530f411130fd7acb88b8251009a0829379aa000000000000000000000000a4ebe1e06dd0bf674b0757cd20b97ee16b00af1b0000000000000000000000009efb09cdfbe41c666f8441aa5f8d84ec8c8f044500000000000000000000000067a95e050d2e4200808a488628d55269ddefc455000000000000000000000000d1c8b1e58c1186597d1897054b738c551ec74bd40000000000000000000000003d446d0d45b7fab5aa7954bd9140b56523ba32f7000000000000000000000000080d263faa8cbd848f0b9b24b40e1f23ea06b3a30000000000000000000000000000000000000000000000000000000000000010000000000000000000000000218b5a7861dbf368d09a84e0dbff6c6ddbf99db800000000000000000000000070bc61658615725a82d5e78ff0066bfcb1b98988000000000000000000000000b976d01275b809333e3efd76d1d31fe9264466d0000000000000000000000000cc29be4ca92d4ecc43c8451fba94c200b83991f600000000000000000000000061317c73d0225b2e37140fb9664d607b450613c6000000000000000000000000c4b732fd121f2f3783a9ac2a6c62fd535fd13fda000000000000000000000000fa0e4f48a369bb3ecbcee0b5119379ea8d1bcf290000000000000000000000007bfb89db2d7217c57c3ad3d4b55826efd17dc2e9000000000000000000000000dbfea8d5822141c13f92caa06eb94d0f3d67c2430000000000000000000000003c4ad65f5b4884397e1f09596c7ac7f8f95b3ff3000000000000000000000000f34ac04a28f7cb5324a167c96b24ade9c742b44f000000000000000000000000ce859e48f6ce9834a119ba04fdc53c1d4f1082a7000000000000000000000000dde59ceec7a2cc8b2bd78199877ba22018966813000000000000000000000000c74ce67bfc623c803d48afc74a09a6ff6b599003000000000000000000000000b5b90f596341127de460465dea9b28b8a6bd1984000000000000000000000000cf4be57aa078dc7568c631be7a73adc1cda992f80000000000000000000000000000000000000000000000000000000000000be000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000826299e0000000000000000000000000000000000000000000000000000000003f5476a0000000000000000000000000000000000000000000000000000000006fc23ac0000000000000000000000000000000000000000000000000000000002cb41780000000000000000000000000000000000000000000000000000004e94914f00000000000000000000000000000000000000000000000000000000000001312d000000000000000000000000000000000000000000000000000000000df84758000000000000000000000000000000000000000000000000000000000000000006000000000000000000000000000000000000000000000000000000000000018000000000000000000000000000000000000000000000000000000000000003a000000000000000000000000000000000000000000000000000000000000005c0000000000000000000000000000000000000000000000000000000000000094000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000001009f2d0a6c4cd968fd18042bb6451f2574fea0a875695fe1fa9529dbea5dd8235e0780117191f1e57787c706dea94dfbafbc0c115e0ad622f053c46829de955b13f6ed381d973b5d843882c496de2faf4f0f2943240c7da500b6126a2b9fa6c1d0dd789d4e970d0f999855f943b44d3fb497d6fe365afca17f6d35b868e516875887a76356bc97b88bb15b5c1d9750290f0324cd62e5046743bbabc900cdcc2d4e91a3af408c56c32b315ba9ff6839625ca6d847bb77cbedab6efc011a926405cb12cc0640aa6c047c51ce0ab4ba5a56e0462745ea92b3e5e7e2339886611e6d1b490fcc4c4399222102251a14699dae7676ab2ba45bf3cbce854b502528fd524ae90be70e97fab367a0a7444bd1157f2ca81e328e854553f2cf239419c96ef0d43a310e03ed3c99b01b43a6b6e6ad1ea73051b15a42ac3f44f18dcd1f1c1852fa6f78d3d940ed1eee3391a8955b723ac5f1a65c6ef0179a762034f55fd9a0aec0fdad1cdecc8c350416ca7b3ae8b9d6d7b51de109360f0e19e8107ed8fb0c3c9fc6b87a6bc917e0c4f8088105e7fa28cbbf70dd44f9fd9e45c5c6c4e7b1a5cb3268e5bb8f07b165ec9cb6f219eb16eee60def6a0225d4f4028f3d25f6e9ba9fedabc9e45919ff3013e16cfa6f65b20acb0c3733f47a0ef5cc5536b02a4da1e5323592f9c86d4ed6f10d07169e32a8896ce4cb1374b2e4d09f388699a5a4ed061000000000000000000000000000000000000000000000000000000000000034f313244334b6f6f57436571324e347442536d7952426461536e6a6a6f7859556a346850734e4b617567773647516477696a616f592c313244334b6f6f5741727331455676667973486b6936794e525a6f6d676e58336b7a427574434769584a517571384d68693756462c313244334b6f6f574761566252356378546e384b656355766a78574d505a7233415642333556784c4675424a43363376764a4d772c313244334b6f6f574878666b636e44653154615846684868747a4233544663716536736b374b4e516573776337434644786832692c313244334b6f6f57414e674b376574716177674b64584d61374d446e6d46566e5447446554414271736e6e6b5756446b544161522c313244334b6f6f574771705975695534756d34384358795a6b6a736f664458524d5166736f34733452584848785a756a444559382c313244334b6f6f5751387331734a6555554b665a5377573957656a623561776b313374524d724e62464d7558486a654e4d7133482c313244334b6f6f57413250716f563733596e34674c774b5a7466376b5a7441573855706954506a45484844595a334862735843382c313244334b6f6f5748765435536764597437465353454e4a47414753395a4a6d316a565137486d534e4e573265587566316f33312c313244334b6f6f575246754855446d37696162507a6e6d37797576736776594e717a4a7a69444d4a527247426d6e5631786d58322c313244334b6f6f574e427047466e5346326d6d5a3877654c56337334487435564756346a756f4b733333577a4b6351334b39704e2c313244334b6f6f57524c464b56376768357a4d524433354872327242757a6f4644436761414c644e59546b4c736350556b36684c2c313244334b6f6f574131513662634c51594b57794b7965727742547857735958474e6e7a366b4c6377596f6b6a44476a697262622c313244334b6f6f5753704c6b7544396234327972466f70724351756a705734676b7254414753574e3745666e756d68556b764b422c313244334b6f6f574b3142554374343274776e3446436f7554364364526b64587a653436625a6d31314e545447777374704257702c313244334b6f6f574a47463953425932484735544c48664a724d7638626f6d3844476a4c32314d417a695236744261367a316a720000000000000000000000000000000000bb44313ca6fb46afbce73ea9aad8891378cfb9712b7d1f3c44c5200f138a48235140111502a59685f3e2d6e02aab340252bf176043df41741f14d57ec470c04d000000000000000000000000000000000000000000000000000000000000006000000000000000000000000000000000000000000000000000000000000000108a5308ef237fc50998b6a3c5fb1d82950000000000000000000000000000000063f5b3e2ce9cda7a27b63078298303d400000000000000000000000000000000963a80a350dadc0dcfee5a982021499400000000000000000000000000000000a53c37a49e05a0918991b3806590b24f000000000000000000000000000000000536030931d300983adc31c8e14f742e000000000000000000000000000000001c7ded5c1905d4d3cf5f25068f310ea0000000000000000000000000000000001f57d0cb757d141fa18ae3377cce432000000000000000000000000000000000f7f6a176a1b38ad779d450294b65a30300000000000000000000000000000000ab7f7f8da991407011d40d908208867d000000000000000000000000000000000a59cb132a6fc955a07dd76bacbcc73400000000000000000000000000000000526f9718f322681dbc9dc8743db94ad9000000000000000000000000000000006e1a28774ddc66595f7c4a2e00014b8c00000000000000000000000000000000845cfec5cb6de0158d5d2fd017df76d50000000000000000000000000000000068699dddd07d58cfb8119ae5386bc31f000000000000000000000000000000002f487bf771ad1f2445902404744d51b9000000000000000000000000000000009fc9294accb3e2711540ef2ceea2105e00000000000000000000000000000000'

ConfigSet_ABI = get_event_abi(abi, 'ConfigSet')
ConfigSet_topic = encode_hex(event_abi_to_log_topic(ConfigSet_ABI)) 
ConfigSet_event = AttributeDict({'data':ConfigSet_data
                                ,'topics': [HexBytes(ConfigSet_topic)]
                                ,'address':'' 
                                ,'logIndex':''
                                ,'transactionIndex':''
                                ,'transactionHash':''
                                ,'blockHash':''
                                ,'blockNumber':''
                             
    })
get_event_data(default_codec, ConfigSet_ABI, ConfigSet_event) # w3.codec->default_codec if we have no rpc connection



AttributeDict({'args': AttributeDict({'previousConfigBlockNumber': 0,
  'configCount': 1,
  'signers': ['0xC01465eBA4FA3A72309374bb67149A8FD14Cb687',
   '0x8d766C898fA75C6a2fc59a078E8C2D2DE12547E2',
   '0xCc1b49B86F79C7E50E294D3e3734fe94DB9A42F0',
   '0x8EB664cD767f12507E7e3864Ba5B7E925090A0E5',
   '0x3650Da40Fe97A93bfC2623E0DcA3899a91Eca0e2',
   '0x0ac6c28B582016A55f6d4e3aC77b64749568Ffe1',
   '0x8316e3Eb7eccfCAF0c1967903CcA8ECda3dF37E0',
   '0x1E15545b23B831fD39e1d9579427DeA61425DD47',
   '0x5007b477F939646b4E4416AFcEf6b00567F5F078',
   '0xbd34530f411130fd7aCB88b8251009A0829379aA',
   '0xA4EBE1e06dd0bf674B0757cd20b97Ee16b00aF1B',
   '0x9eFB09cDfbe41C666F8441AA5F8D84eC8c8f0445',
   '0x67a95e050d2E4200808A488628d55269dDeFC455',
   '0xD1C8b1e58C1186597D1897054b738c551ec74BD4',
   '0x3d446d0d45B7fAb5Aa7954Bd9140b56523BA32f7',
   '0x080D263FAA8CBd848f0b9B24B40e1f23EA06b3A3'],
  'transmitters': ['0x218B5a7861dBf368D09A84E0dBfF6C6DDbf99DB8',
   '0x70bC61658615725A82D5e78FF0066BFCb1b98988',
